In [5]:
import os
import requests
import pandas as pd
import torch
import  numpy as np

In [6]:
print(f"{torch.__version__}")

2.2.1+cu118


In [7]:
uoh_csv_path='data/uoh.csv'
uoh_excel_path='data/uoh.xlsx'
uoh_df=pd.read_excel(uoh_excel_path,dtype=str)

In [8]:
uoh_df.columns,uoh_df.columns.dtype

(Index(['Name', 'Description', 'Example pests controlled',
        'Example applications', 'Efficacy & activity',
        'Appearance and life cycle', 'Availability status',
        'Introduction & key dates', 'Taxonomic classification',
        'UK COPR regulatory status', 'Date COPR inclusion expires',
        'UK LERAP status', 'EC Regulation 1107/2009 status',
        'Dossier rapporteur/co-rapporteur',
        'Date EC 1107/2009 inclusion expires',
        'EU Candidate for substitution (CfS)', 'Listed in EU database',
        'Approved for use (✓) under EC 1107/2009 in the following EU Member States',
        'Approved for use (✓) under EC 1107/2009 by Mutual Recognition of Authorisation and/or national regulations in the following EEA countries',
        'Known to be used in the following countries', 'Isomerism',
        'Chemical formula', 'Canonical SMILES', 'Isomeric SMILES',
        'International Chemical Identifier key (InChIKey)',
        'International Chemical Identifie

In [9]:
test_df=uoh_df.head(50)
use_cols = ['Name',
            'Description',
            'Canonical SMILES',
            'Isomeric SMILES',
            'Biopesticide type',
            'Mode of action',
            'Substance source',
            'Target pests',
            'Target host',
            'IUPAC name',
            'CAS name',
            'Related substances & organisms',
            'Formulation and application details']
from collections import defaultdict
test_dict=dict.fromkeys(test_df['CAS RN'])
for key in test_dict.keys():
    test_dict[key]={}
for index,row in test_df.iterrows():
    for col in use_cols:
        if test_df[col].isnull().iloc[index]:
            continue
        if col!='Related substances & organisms':
            test_dict[row['CAS RN']].update({col:row[col]})
        else:
            test_dict[row['CAS RN']].update({col:row[col].strip().split('\n')})
test_dict 

{'59920-31-9': {'Name': '1,2,4-trimethoxybenzene',
  'Description': 'A naturally occurring sugar anologue that has nematicide activity currently being explored for its potential as a biocontrol agent',
  'Canonical SMILES': 'C(C1C(C(C(N1)CO)O)O)O',
  'Isomeric SMILES': 'C([C@@H]1[C@H]([C@@H]([C@H](N1)CO)O)O)O',
  'Biopesticide type': 'Insecticide, Nematicide',
  'Mode of action': 'Unclear but thought to relate to glycosidase inhibition',
  'Substance source': 'Isolated from tropical legumes (e.g. Lonchocarpus, Derris)',
  'Target pests': 'Wide variety of soil-borne nematodes',
  'Target host': 'Wide variety of crops including vegetables, cotton, tobacco, coffee, tea, bananas',
  'IUPAC name': '2,5-dideoxy-2,5imino-D-mannitol',
  'CAS name': '(2R,5R)-dihydroxymethyl-(3R,4R)-dihydroxypyrrolidine',
  'Related substances & organisms': ['rotenone'],
  'Formulation and application details': 'Usually formulated for application as a foliar spray'},
 '38421-90-8': {'Name': '1,2-octanediol',
  '

In [10]:
pesticides=list(test_dict.keys())

In [11]:
# 有些单元分割是 ;  有些是 ,  先把 ; 都替换成 , 再分割字符串
test_df["Target host"] = test_df["Target host"].str.replace(";", ",")
test_df["Target host"] = test_df["Target host"].str.replace("Wide variety of ", "")
test_df["Target host"] = test_df["Target host"].str.replace("Wide range of", "")
test_df["Target host"] = test_df["Target host"].str.replace("A variety of different ", "")
test_df["Target host"] = test_df["Target host"].str.replace("A wide variety of", "")
test_df["Target host"] = test_df["Target host"].str.replace("A wide range of ", "")
test_df["Target host"] = test_df["Target host"].str.replace("including", ",")
test_df["Target host"] = test_df["Target host"].str.replace("such as", ",")
test_df["Target host"] = test_df["Target host"].str.replace("&", ",")
test_df["Target host"] = test_df["Target host"].str.replace(" and", ",")
test_df["Target host"] = test_df["Target host"].str.replace("  ", ",")
test_df["Target host"] = test_df["Target host"].str.split(", ")
for i in test_df['Target host']:
    for j in i:
        j = j.strip()
test_df['Target host']

/tmp/ipykernel_448108/1759917121.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["Target host"] = test_df["Target host"].str.replace(";", ",")
/tmp/ipykernel_448108/1759917121.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["Target host"] = test_df["Target host"].str.replace("Wide variety of ", "")
/tmp/ipykernel_448108/1759917121.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



0     [crops , vegetables, cotton, tobacco, coffee, ...
1           [crops , top fruit, stone fruit, tree nuts]
2           [crops , top fruit, stone fruit, tree nuts]
3                                          [Pine trees]
4           [crops , top fruit, stone fruit, tree nuts]
5                                [Tomatoes, aubergines]
6                                         [Grape vines]
7                                 [Peas, other legumes]
8                     [Top fuit, Stone fruit, Currants]
9                                     [Crop protection]
10                                       [Mainly vines]
11                    [Top fuit, Stone fruit, Currants]
12                              [Top fuit, Stone fruit]
13                                             [Apples]
14    [crops , lucerne, asparagus, vegetables, salad...
15    [fruit crops , top fruit, stone fruit, some so...
16                                          [Fir Trees]
17    [Top fruit, Stone fruit, Nuts , other tree

In [12]:
# 有些单元分割是 ;  有些是 ,  先把 ; 都替换成 , 再分割字符串
test_df['Target pests'] = test_df['Target pests'].str.replace(';', ',')
test_df['Target pests'] = test_df['Target pests'].str.replace('  ', ',')
test_df['Target pests'] = test_df['Target pests'].str.replace('Various insects belong to the', '')
test_df['Target pests'] = test_df['Target pests'].str.replace(
    'Wide variety of', '')
test_df['Target pests'] = test_df['Target pests'].str.replace('A wide range', '')
test_df['Target pests'] = test_df['Target pests'].str.replace(
    'A variety of caterpillars', '')
test_df['Target pests'] = test_df['Target pests'].str.replace(
    'A wide range of', '')
test_df['Target pests'] = test_df['Target pests'].str.replace('including', '')
test_df['Target pests'] = test_df['Target pests'].str.replace('such as', ',')
test_df['Target pests'] = test_df['Target pests'].str.replace('&', ',')
test_df['Target pests'] = test_df['Target pests'].str.replace(' and', ',')
test_df['Target pests'] = test_df['Target pests'].str.split(', ')
for  i in test_df['Target pests']:
    for j in i:
        j=j.strip()
test_df['Target pests']

/tmp/ipykernel_448108/1754555870.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Target pests'] = test_df['Target pests'].str.replace(';', ',')
/tmp/ipykernel_448108/1754555870.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Target pests'] = test_df['Target pests'].str.replace('  ', ',')
/tmp/ipykernel_448108/1754555870.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

0                               [ soil-borne nematodes]
1     [ Lepidoptera order  the codling moth (Cydia p...
2     [ Lepidoptera order  the oriental fruit moth (...
3     [Various lepidopteran pests  the Eastern pine ...
4     [ Lepidoptera order  the codling moth (Cydia p...
5            [Tomato pinworm (Keiferia lycopersicella)]
6           [European Grapevine Moth (Lobesia botrana)]
7                          [Pea moth (Cydia nigricana)]
8     [Leopard moths (Zeuzera pyrina), Lesser peacht...
9       [Light brown Apple Moth (Epiphyas postvittana)]
10    [ Lepidoptera order  the European grapevine mo...
11    [Leopard moths (Zeuzera pyrina), Lesser peacht...
12    [Leopard moths (Zeuzera pyrina), Lesser peacht...
13      [Apple leaf miner (Phyllonorycter ringoniella)]
14                  [Beet armyworm (Spodoptera exigua)]
15    [ Lepidoptera order  the tea tortrix (Hormona ...
16    [Douglas fir tussock moth (Orgyia pseudotsugata)]
17    [Oriental fruit moths (Grapholitha molesta

In [13]:
pests=test_df['Target pests'].explode().unique()
pests = [str(item).strip() if item is not None else '' for item in pests]
pests

['soil-borne nematodes',
 'Lepidoptera order  the codling moth (Cydia pomonella)',
 'pandemis leafroller (Pandemis pyrusana)',
 'the fruit tree leafroller (Archips argyrospilus)',
 'Lepidoptera order  the oriental fruit moth (Grapholitha molesta)',
 'the macadamia nut borer (Cryptophlebia ombrodelta)',
 'Various lepidopteran pests  the Eastern pine shoot borer (Eucosma gloriola)',
 'the European pine shoot moth (Rhyacionia buoliana)',
 'European corn borer (Ostrinia nubilalis',
 'Tomato pinworm (Keiferia lycopersicella)',
 'European Grapevine Moth (Lobesia botrana)',
 'Pea moth (Cydia nigricana)',
 'Leopard moths (Zeuzera pyrina)',
 'Lesser peachtree borer (Synathedon pictipes)',
 'Greater peachtree borer (Synathedon exitisoa)',
 'currant clearwing moth (Synanthedon tipuliformis)',
 'Light brown Apple Moth (Epiphyas postvittana)',
 'Lepidoptera order  the European grapevine moth,(Lobesia botrana)',
 'Apple leaf miner (Phyllonorycter ringoniella)',
 'Beet armyworm (Spodoptera exigua)',


In [14]:
hosts=test_df['Target host'].explode().unique()
hosts = [str(item).strip() if item is not None else '' for item in hosts]
hosts

['crops',
 'vegetables',
 'cotton',
 'tobacco',
 'coffee',
 'tea',
 'bananas',
 'top fruit',
 'stone fruit',
 'tree nuts',
 'Pine trees',
 'Tomatoes',
 'aubergines',
 'Grape vines',
 'Peas',
 'other legumes',
 'Top fuit',
 'Stone fruit',
 'Currants',
 'Crop protection',
 'Mainly vines',
 'Apples',
 'lucerne',
 'asparagus',
 'salads',
 'strawberries',
 'fruit crops',
 'some soft fruit',
 'cranberries',
 'nuts',
 'vines',
 'Fir Trees',
 'Top fruit',
 'Nuts',
 'other tree fruit',
 'crops',
 'pine forests',
 'Olives',
 'Citrus',
 'legumes',
 'beets',
 'soft fruit',
 'stone fruit',
 'Cotton',
 'ornamental',
 'agricultrual crops']

In [15]:
# test_df.set_index('CAS RN',inplace=True)
# # 将索引映射成连续值
# mapping = {index: i for i, index in enumerate(test_df.index.unique())}
# mapping

In [16]:
test_df['Related substances & organisms']=test_df['Related substances & organisms'].str.strip().str.split('\n')
test_df['Related substances & organisms']

/tmp/ipykernel_448108/3429370798.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Related substances & organisms']=test_df['Related substances & organisms'].str.strip().str.split('\n')


0                                            [rotenone]
1     [(Z)-tetradec-9-enyl acetate, (E)-dodec-8-enyl...
2     [(Z)-tetradec-9-enyl acetate, (E)-dec-5-enyl a...
3                            [(Z)-dodec-9-enyl acetate]
4     [(Z)-11-tetradecenyl acetate, (E)-dodec-8-enyl...
5                                                   NaN
6                            [(E)-dodec-9-enyl acetate]
7                                                   NaN
8                                                   NaN
9                        [(E)-tetradec-11-enyl acetate]
10    [(E)-tetradec-11-enyl acetate, (E)-dodec-8-eny...
11                                                  NaN
12                                                  NaN
13                                                  NaN
14                                [(Z)-tetradec-9-enol]
15    [(E)-tetradec-11-enyl acetate, (E)-dodec-8-eny...
16                                                  NaN
17              [8-dodecenol, (E)-dodec-8-enyl a

In [17]:
test_df['Related substances & organisms'].describe()

count                             27
unique                            13
top       [(E)-dodec-9-enyl acetate]
freq                               5
Name: Related substances & organisms, dtype: object

In [18]:
from sentence_transformers import SentenceTransformer


class SequenceEncoder:
    def __init__(self, model_path='models/all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_path, device=device)
        self.max_length = 510  # 留有一定的缓冲

    @torch.no_grad()
    def test_encode(self, text: str):
        x = self.model.encode(text, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()

    def test_decoed(self, text):
        x = self.model.decode(text)
        return x

    @torch.no_grad()
    def __call__(self, df: pd.DataFrame):
        x = self.model.encode(df.values, show_progress_bar=True,
                                convert_to_tensor=True, device=self.device)
        print(x.shape)
        return x.cpu()
    def encode_list(self,seqs):
        x = self.model.encode(seqs, show_progress_bar=True,
                                convert_to_tensor=True, device=self.device)
        print(x.shape)
        return x.cpu()

/home/zdw/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
cols=[
    # 'Name',
    'Description',
    'Canonical SMILES',
    # 'Isomeric SMILES',
    # 'Biopesticide type',
    'Mode of action',
    'Substance source',
    # 'Target pests',
    # 'Target host',
    # 'IUPAC name',
    'CAS name',
    # 'Related substances & organisms',
    # 'Formulation and application details'
]
encoder=SequenceEncoder()


No sentence-transformers model found with name models/all-MiniLM-L6-v2. Creating a new one with MEAN pooling.


In [20]:
pesticide_features = [encoder(test_df[col]) for col in cols]
pesticide_features = torch.cat(pesticide_features, dim=-1)
pesticide_features.shape

Batches: 100%|██████████| 2/2 [00:00<00:00,  3.20it/s]


torch.Size([50, 768])


Batches: 100%|██████████| 2/2 [00:00<00:00, 69.62it/s]


torch.Size([50, 768])


Batches: 100%|██████████| 2/2 [00:00<00:00, 100.63it/s]


...

Batches: 100%|██████████| 2/2 [00:00<00:00, 65.97it/s]


torch.Size([50, 768])


Batches: 100%|██████████| 2/2 [00:00<00:00, 78.30it/s]

torch.Size([50, 768])


torch.Size([50, 3840])

In [21]:
pests_features = encoder.encode_list(pests)
pests_features.shape

Batches: 100%|██████████| 2/2 [00:00<00:00, 93.18it/s]

torch.Size([39, 768])


torch.Size([39, 768])

In [22]:
hosts_features = encoder.encode_list(hosts)
hosts_features.shape

Batches: 100%|██████████| 2/2 [00:00<00:00, 114.06it/s]

torch.Size([46, 768])


torch.Size([46, 768])

In [23]:
from torch_geometric.data import  HeteroData
data = HeteroData()
data['pesticide'].x= pesticide_features
data['pesticide'].num_nodes=len(pesticides)
data['pests'].x=pests_features
data['pests'].num_nodes=len(pests)
data['hosts'].x=hosts_features
data['hosts'].num_nodes=len(hosts)
data.stores, data.node_types

([{},
  {'x': tensor([[-0.3940,  0.4294,  0.1966,  ..., -0.3859,  0.4937,  0.4087],
          [-0.2171,  0.6521, -0.0270,  ..., -0.3866,  0.6243,  0.6264],
          [-0.2171,  0.6521, -0.0270,  ..., -0.2876,  0.4204,  0.7546],
          ...,
          [-0.2298,  0.8230, -0.1694,  ..., -0.2677,  0.4393,  0.6088],
          [-0.2692,  0.7679, -0.1946,  ..., -0.2895,  0.4354,  0.6210],
          [-0.3581,  0.8647, -0.1952,  ..., -0.4187,  0.5079,  0.6748]]), 'num_nodes': 30},
  {'x': tensor([[-0.0429,  0.7376, -0.2077,  ..., -0.1970, -0.0796,  0.2288],
          [-0.1968,  0.4736, -0.2782,  ..., -0.0601,  0.3771,  0.1939],
          [-0.0858,  0.2165, -0.3323,  ..., -0.2038,  0.3364,  0.2674],
          ...,
          [-0.0111,  0.5167, -0.0365,  ..., -0.0229,  0.2377,  0.3349],
          [-0.1508,  0.4836, -0.0190,  ...,  0.0277,  0.2545,  0.0629],
          [-0.0217,  0.0638, -0.1636,  ..., -0.2587,  0.1731,  0.0731]]), 'num_nodes': 39},
  {'x': tensor([[ 0.3540,  0.4603,  0.1796,  ...

In [24]:
t=[]
for index,row in test_df.iterrows():
    t.append((index,row['CAS RN'],row['Target pests'],row['Target host']))
t

[(0,
  '59920-31-9',
  [' soil-borne nematodes'],
  ['crops ', 'vegetables', 'cotton', 'tobacco', 'coffee', 'tea', 'bananas']),
 (1,
  '38421-90-8',
  [' Lepidoptera order  the codling moth (Cydia pomonella)',
   'pandemis leafroller (Pandemis pyrusana)',
   'the fruit tree leafroller (Archips argyrospilus)'],
  ['crops ', 'top fruit', 'stone fruit', 'tree nuts']),
 (2,
  '38363-29-0',
  [' Lepidoptera order  the oriental fruit moth (Grapholitha molesta)',
   'the macadamia nut borer (Cryptophlebia ombrodelta)'],
  ['crops ', 'top fruit', 'stone fruit', 'tree nuts']),
 (3,
  '35148-19-7',
  ['Various lepidopteran pests  the Eastern pine shoot borer (Eucosma gloriola)',
   'the European pine shoot moth (Rhyacionia buoliana)'],
  ['Pine trees']),
 (4,
  '33189-72-9',
  [' Lepidoptera order  the codling moth (Cydia pomonella)',
   'European corn borer (Ostrinia nubilalis',
   'pandemis leafroller (Pandemis pyrusana)',
   'the fruit tree leafroller (Archips argyrospilus)'],
  ['crops ', 't

In [25]:
# 假设以下是害虫、植物宿主和农药到索引的映射（根据你的集合构建）
pest_to_idx = {pest: idx for idx, pest in enumerate(pests)}
host_to_idx = {host: idx for idx, host in enumerate(hosts)}
pesticide_to_idx = {pesticide: idx for idx,
                    pesticide in enumerate(test_df['CAS RN'].unique())}

In [26]:
# 初始化边索引列表
prevents_edges = [[], []]  # 农药到害虫
affects_edges = [[], []]  # 害虫到植物宿主
heal_edges = [[], []]  # 农药到植物宿主

for idx, row in test_df.iterrows():
    pesticide_idx = pesticide_to_idx[row['CAS RN']]
    for pest in row['Target pests']:
        pest_idx = pest_to_idx[pest.strip()]
        if pest in test_df.loc[idx,'Target pests']:
            prevents_edges[0].append(pesticide_idx)
            prevents_edges[1].append(pest_idx)
        for host in row['Target host']:
            host_idx = host_to_idx[host.strip()]
            if host in test_df.loc[idx,'Target host']:
                affects_edges[0].append(pest_idx)
                affects_edges[1].append(host_idx)
                heal_edges[0].append(pesticide_idx)
                heal_edges[1].append(host_idx)

# 将边索引添加到HeteroData对象
data['pesticide', 'prevents', 'pests'].edge_index = torch.tensor(
    prevents_edges, dtype=torch.long)
data['pests', 'affects', 'hosts'].edge_index = torch.tensor(
    affects_edges, dtype=torch.long)
data['pesticide', 'heal', 'hosts'].edge_index = torch.tensor(
    heal_edges, dtype=torch.long)

In [27]:
import pickle
with open('data/hetero_graph.pkl', 'wb') as f:
    pickle.dump(data, f)

In [28]:
data

HeteroData(
  pesticide={
    x=[50, 3840],
    num_nodes=30,
  },
  pests={
    x=[39, 768],
    num_nodes=39,
  },
  hosts={
    x=[46, 768],
    num_nodes=46,
  },
  (pesticide, prevents, pests)={ edge_index=[2, 97] },
  (pests, affects, hosts)={ edge_index=[2, 299] },
  (pesticide, heal, hosts)={ edge_index=[2, 299] }
)